(Work in Progress)

# Recommending Top-N movies - Part 2: Testing recommendation algorithms

In part 1 of this project, we looked at the MovieLens data set and implemented algorithms to recommend movies based off of this. We took a brief look at some of these algorithms parameters and differences in runtime. In this part of the project, we will use various metrics to test the accuracy and validity of the recommended movies using a train-test-validation split of the reduced data set.

To do this we'll use three new classes: SplitData, Metrics, and Tester which split the ratings pivot data frame, run metrics on the recommendations, and run tests over multiple users respectively. More documentation for these classes can be found in the MovieLensData.py and Tester.py files, with further information on the algorithms in the Algorithms.py file.

---------------

## Preparing the Data

- Creating the ratings pivot data frame
- Splitting the data for testing
- Building the models

We'll start by importing the required packages and building the ratings pivot data frame from a reduced set of data. As discussed in the part 1, we're sampling users with rating counts between 200-6000 and limiting movies to those with more than 100 ratings to avoid biases from outliers and to work within the constraints of computing power available. However, as the tests are very computationally intensive, I will initially be reducing this further by taking a random sample of 10% of the reduced userIDs.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from time import perf_counter
from tqdm.auto import tqdm
import sys
import importlib
import random
import MovieLensData
import Algorithms
import Tester

import warnings
warnings.filterwarnings('ignore')
sns.set()

In [2]:
ml = MovieLensData.MovieLensData()
userIDs = ml.filterIDs('userId', minRatings=200, maxRatings=6000)
movieIDs = ml.filterIDs('movieId', minRatings=100)
userIDs = random.sample(userIDs, int(0.1 * len(userIDs)))
ml.reduce(userIDs, 'userId', 'ratings')
ml.reduce(movieIDs, 'movieId', 'movies')

ratingsData = ml.buildPivot(printStats=True)

Loading data... Done 
Building movies/ratings pivot df... Done 
3528 / 283228 users retained (1.25%)
10500 / 58098 movies retained (18.07%)
1679766 / 27753444 ratings retained (6.05%)
95.47% sparsity


The SplitData class splits up the ratings pivot data frame into a trainSet and testSet_full. testSet_full is then split in two ways: one into a a test-validation split where every user in testSet_full has a percentage or number of ratings split off into the validation set, and one that takes away just one rating from each user for leave-one-out cross validation (LOO-CV). The size of testSet_full and validation sets are governed by the testSize and validationSize parameters respectively. As the data is relatively large, we afford to keep more users in the train set, so 20% splits will be used for both the testSize and validationSize parameters.

The ratings in the test set are used to generate recommendations, with the two validation sets used to check the quality of recommendations against various metrics. We can ensure the same splits occur each time via the randomState parameter for consistency in results reporting.

In [3]:
train, test, validation, LOO_test, LOO_dropped = MovieLensData.SplitData(ratingsData).buildAll(testSize=0.2, 
                                                                             validationSize=0.2, randomState=20)

Building train/test/validation split by row... Done 
Building LeaveOneOut-CrossValidation data... Done 


As seen in the previous part, we can easily build each model by calling the buildModel method in the Algorithm class. This time, we initialise the method by passing the train and test set data so the program knows which set to build the model from, and which to use to look up ratings for testing. We will also need to call the buildMatrix method to generate the sparse matrix for building the KNN models.

However, this time we will not need to generate a similarity matrix for the users. As these models are built off of the train set and we will not be testing any f the userIDs in this set, we do not need to build a user similarity matrix for the CF algorithm and can simply pass 'None' as the model parameter when testing this algorithm.

In [4]:
algo = Algorithms.Algorithms(ml, train, test)
ratingsSparse = algo.buildMatrix()
printStatus = True

CF_itemModel = algo.buildModel(modelType='CF', matrix='item', printStatus=printStatus)
KNN_itemModel = algo.buildModel(modelType='KNN', matrix=ratingsSparse.transpose(), printStatus=printStatus)
KNN_userModel = algo.buildModel(modelType='KNN', matrix=ratingsSparse, printStatus=printStatus)
#SVD_model = algo.buildModel(modelType='SVD', printStatus=printStatus)

Initialising object... Done 
Building sparse matrix... Done 

Building CF model
Correlating ratings for all movies... Done 
Correlating genres for all movies... Done 
Correlating years for all movies... Done 
Generating combined correlation... Done 
Done. Time: 15.203528900000002s

Building KNN model
Done. Time: 0.022894700000001933s

Building KNN model
Done. Time: 0.0064758999999980915s


## Testing Demonstration and Metrics

- Adding algorithms for testing
- Running basic and parameter tests
- MAE, RMSE, Coverage, Diversity, Novelty, Hit Rates (Validation, LOO-CV, Cumulative, Mean Reciprocal, Actual Rating)

### Working with the Tester class

The Metrics class contains the various metrics we'll be using the test the quality of our results. This takes the validation and LOO-CV sets to check the results against, a filename for the csv that stores the results, as well as various parameters for testing (discussed below).

This object is then passed to the Tester class to create our test object. This object has two primary methods for testing, runBasicTest and runParameterTest, as well as two methods for adding and removin algorithms. All we need to do to set up tests is to instantiate the tester object, and pass the various algorithms' names, method to call, model and all other parameters to control for this test. The parameters we control will be discussed in greater detail when discussing results later in this notebook.

Here we'll also add a random control algorithm which simply generates random similarity scores and random rating precitions for every user. The randomRatings parameter of this method determines whether or not the rating predictions are a random number between 0-5 independent of the similarity scores, or if the rating predictions are a multiple of the randomly generated similarity scores.

In [7]:
filename = 'ML_MetricsData_sample'
evaluator = Tester.Metrics(validation, LOO_dropped, topN=10, moviesPerPage=5, thresholdRating=3.0, csvName=filename)
tester = Tester.Tester(evaluator)

neighbours = 100
sample = 100
thresh = 2.0

tester.addAlgorithm('Item-Based CF', algo.itemBased, model=CF_itemModel, modelType='CF', neighbours=neighbours,
                    sample=sample, threshold=thresh, predict='calc')
tester.addAlgorithm('User-Based CF', algo.userBased, model=None, modelType='CF', neighbours=neighbours,
                    sample=sample, threshold=thresh, predict='calc')
tester.addAlgorithm('Item-Based KNN', algo.itemBased, model=KNN_itemModel, modelType='KNN', neighbours=neighbours,
                    sample=sample, threshold=thresh, predict='calc')
tester.addAlgorithm('User-Based KNN', algo.userBased, model=KNN_userModel, modelType='KNN', neighbours=neighbours,
                    sample=sample, threshold=thresh, predict='calc')
#tester.addAlgorithm('SVD', algo.SVD, model=SVD_model, sample=sample, pred='calc')
tester.addAlgorithm('Random Control', algo.random, randomRatings=True)

ML_MetricsData.csv already exists. Update this file (u) or overwrite (o)? u
Added Item-Based CF
Added User-Based CF
Added Item-Based KNN
Added User-Based KNN
Added Random Control


# TEST

In [ ]:
filename = 'ML_MetricsData'
evaluator = Tester.Metrics(validation, LOO_dropped, topN=30, moviesPerPage=5, thresholdRating=3.0, csvName=filename)
tester = Tester.Tester(evaluator)

neighbours = 100
sample = 100
thresh = 2.0

tester.addAlgorithm('Item-Based CF', algo.itemBased, model=CF_itemModel, modelType='CF', neighbours=neighbours,
                    sample=sample, threshold=thresh, predict='calc')
tester.addAlgorithm('Item-Based KNN', algo.itemBased, model=KNN_itemModel, modelType='KNN', neighbours=neighbours,
                    sample=sample, threshold=thresh, predict='calc')
tester.addAlgorithm('User-Based CF', algo.userBased, model=None, modelType='CF', neighbours=neighbours,
                    sample=sample, threshold=thresh, predict='calc')
tester.addAlgorithm('User-Based KNN', algo.userBased, model=KNN_userModel, modelType='KNN', neighbours=neighbours,
                    sample=sample, threshold=thresh, predict='calc')

In [ ]:
parameter = 'sample'
pRange = np.arange(150, 2050, 50)

#tester.runParameterTest(test, testAlgo='Item-Based KNN', param=parameter, pRange=pRange, printResults=False)
tester.runParameterTest(test, testAlgo='User-Based CF', param=parameter, pRange=pRange, printResults=False)
tester.runParameterTest(test, testAlgo='User-Based KNN', param=parameter, pRange=pRange, printResults=False)

In [ ]:
parameter = 'neighbours'
pRange = np.arange(150, len(train.columns), 50)

tester.runParameterTest(test, testAlgo='Item-Based CF', param=parameter, pRange=pRange, printResults=False)

parameter = 'neighbours'
pRange = np.arange(200, len(train.index), 200)

tester.runParameterTest(test, testAlgo='User-Based CF', param=parameter, pRange=pRange, printResults=False)
tester.runParameterTest(test, testAlgo='User-Based KNN', param=parameter, pRange=pRange, printResults=False)

With the algorithms loaded, we can now run a simple test using the test set. The runBasicTest takes a few key arguements: 
- testData - the test set
- testAlgo - the name of the algorithm from the dictionary of loaded algorithms to test 
- sampleTest - the number of users from the test to sample
- param, pValue - the paramater and its value to modify from the stored parameters within the object

Here we run a simply test on the above using the stored algorithms for 10 randomly sample users in the test set.

In [26]:
tester.runBasicTest(test, sampleTest=10)

Testing Item-Based CF:   0%|          | 0/10 [00:00<?, ? Users/s]

Testing User-Based CF:   0%|          | 0/10 [00:00<?, ? Users/s]

Testing Item-Based KNN:   0%|          | 0/10 [00:00<?, ? Users/s]

Testing User-Based KNN:   0%|          | 0/10 [00:00<?, ? Users/s]

Testing Random Control:   0%|          | 0/10 [00:00<?, ? Users/s]

The LeaveOneOut cross validation metric uses a different set of ratings per user and is therefore needs to be identified when initialising tests. We can define this by setting LOOCV to True. This usually defaults to False, testing the results against the validation HitRate metric instead (metric types discussed below).

In [27]:
tester.runBasicTest(LOO_test, LOOCV=True, sampleTest=10)

Testing Item-Based CF:   0%|          | 0/10 [00:00<?, ? Users/s]

Testing User-Based CF:   0%|          | 0/10 [00:00<?, ? Users/s]

Testing Item-Based KNN:   0%|          | 0/10 [00:00<?, ? Users/s]

Testing User-Based KNN:   0%|          | 0/10 [00:00<?, ? Users/s]

Testing Random Control:   0%|          | 0/10 [00:00<?, ? Users/s]

The runParameterTest method allows us to iterate through a range of values for a given parameter. The parameter is passed to the runBasicTest to run tests across the users present in the test set. This allows us to see the effect of changing parameters within the methods, with the aim of fine-tuning the algorithms. The parameters for this function are much the same as the runBasicTest method, with pValue now replaced with pRange as the range of parameters to test.

Here we run a basic demonstration on the 'pred' parameter which changes the rating prediction algorithm (discussed in greater in the results).

In [28]:
parameter = 'predict'
pRange = ['rand', 'calc', 'sims', 'norm_sims']
sampleTest = 10

tester.runParameterTest(test, param=parameter, pRange=pRange, sampleTest=sampleTest, printResults=False)

Parameter Testing:   0%|          | 0/4 [00:00<?, ?Parameter/s]

Testing Item-Based CF:   0%|          | 0/10 [00:00<?, ? Users/s]

Testing User-Based CF:   0%|          | 0/10 [00:00<?, ? Users/s]

Testing Item-Based KNN:   0%|          | 0/10 [00:00<?, ? Users/s]

Testing User-Based KNN:   0%|          | 0/10 [00:00<?, ? Users/s]

...

Testing Item-Based CF:   0%|          | 0/10 [00:00<?, ? Users/s]

Testing User-Based CF:   0%|          | 0/10 [00:00<?, ? Users/s]

Testing Item-Based KNN:   0%|          | 0/10 [00:00<?, ? Users/s]

Testing User-Based KNN:   0%|          | 0/10 [00:00<?, ? Users/s]

Testing Random Control:   0%|          | 0/10 [00:00<?, ? Users/s]

By calling the readCSV method in the evaluator class, we can see the results we have just generated ready for analysis.

In [6]:
evaluator.readCSV(filename).head(20)

,Algorithm,Top-N,MoviesPerPage,ThresholdRating,TotalIDs,ParameterTest,ParameterValue,OtherParameters,TotalTime/s,MeanTime/s,MAE,RMSE,Coverage,Diversity,Novelty,LOOCV_HR,Validation_HR,Cumulative_HR,MeanReciprocal_HR,ActualRating_HR
0,Item-Based CF,10,5,3,10,NaN,NaN,"{'neighbours': 100, 'sample': 100, 'threshold'...",1.093928,0.109393,0.590087,0.646977,0.306962,0.344359,0.076681,NaN,0.21,0.21,0.175,"{2.0: 0.000819672131147541, 3.0: 0.00880455407..."
1,User-Based CF,10,5,3,10,NaN,NaN,"{'neighbours': 100, 'sample': 100, 'threshold'...",13.605833,1.360583,0.349836,0.391245,0.221848,0.438315,0.004450,NaN,0.17,0.17,0.120,"{3.5: 0.0029213955443463642, 4.0: 0.0075286138..."
2,Item-Based KNN,10,5,3,10,NaN,NaN,"{'neighbours': 100, 'sample': 100, 'threshold'...",60.645003,6.064500,0.677091,0.679859,0.111914,0.179893,0.003070,NaN,0.10,0.10,0.075,"{3.0: 0.0032903643050143315, 4.0: 0.0010526315..."
3,User-Based KNN,10,5,3,10,NaN,NaN,"{'neighbours': 100, 'sample': 100, 'threshold'...",6.682255,0.668226,0.753723,0.803360,0.122867,0.454609,0.009551,NaN,0.26,0.22,0.225,"{1.5: 0.0006134969325153375, 2.0: 0.0021276595..."
4,Random Control,10,5,3,10,NaN,NaN,{'randomRatings': True},0.162845,0.016285,0.000000,0.000000,1.000000,0.000531,0.543995,NaN,0.00,0.00,0.000,{}
5,Item-Based CF,10,5,3,10,NaN,NaN,"{'neighbours': 100, 'sample': 100, 'threshold'...",1.104494,0.110449,0.564364,0.671683,0.280952,0.343810,0.071127,0.0,NaN,0.22,0.170,"{2.0: 0.002173913043478261, 3.0: 0.00653517759..."
6,User-Based CF,10,5,3,10,NaN,NaN,"{'neighbours': 100, 'sample': 100, 'threshold'...",13.751851,1.375185,0.365606,0.399532,0.216019,0.385336,0.007110,0.0,NaN,0.15,0.120,"{3.0: 0.002000788488074118, 3.5: 0.00112359550..."
7,Item-Based KNN,10,5,3,10,NaN,NaN,"{'neighbours': 100, 'sample': 100, 'threshold'...",63.217344,6.321734,0.404854,0.404854,0.147590,0.141935,0.001882,0.0,NaN,0.09,0.070,"{3.0: 0.00196078431372549, 4.0: 0.004290932799..."
8,User-Based KNN,10,5,3,10,NaN,NaN,"{'neighbours': 100, 'sample': 100, 'threshold'...",6.825581,0.682558,1.038975,1.085616,0.150381,0.355712,0.007172,0.0,NaN,0.23,0.175,"{1.0: 0.0011627906976744186, 1.5: 0.0029171766..."
9,Random Control,10,5,3,10,NaN,NaN,{'randomRatings': True},0.159490,0.015949,0.000000,0.000000,1.000000,0.000485,0.490019,0.0,NaN,0.00,0.000,{}


### Metrics and the Metrics class

The recommendations for each userID are passed to the an instantiate object of the Metrics class for evaluation. It runs various tests and saves the results to a csv file for later analysis. It requires a few parameters upon initialisation to run the tests: 
- Top-N: the top number of recommendations to consider (used by all metrics bar coverage)
- moviesPerPage: the number of movies that will appear per page for the end-user, used only by the meanReciprocalHR metric)
- thresholdRating: the minimum rating to be considered by the cumulativeHR metric

Below is a full list and description of the metrics used for testing.

#### MAE

Error on the predicted ratings from actual ratings. Calculated by taking the absolute difference between the predicted and actual ratings for every hit and finding the mean of these errors. Can range from 0 to infinite. Lower values show a more accurate set of rating predictions.

#### RMSE

Error on the predicted ratings from actual ratings. RMSE gives a higher weight to larger errors and an indication of the increase in variance of the frequency distribution of error magnitudes. Due to this, it will always be equal to or greater than MAE.

RMSE is found by squaring the difference between predicted and actual ratings for hits, finding the mean of these and finally square rooting. Can range from 0 to infinite. Lower values show a more accurate set of rating predictions, with values similar to the MAE value showing less variation of the distribution of errors.

#### Coverage

The number of total movies from the data set the algorithm is able to recommend to the user. Ignores Top-N, dividing the total number of movies in the results by the movies available in the training set. Expressed as a percentage in decimal form ranging from 0 to 1.

Higher coverage values are preferred to ensure the algorithm is accessing the full extent of the items available.

#### Diversity

How diverse the Top-N recommendations are i.e., are the Top-N movies very closely related (low diversity) or not (high diversity). Found finding the mean of the similarities of the Top-N movies (S) and subtracting from 1 (1-S). Expressed as a percentage in decimal form ranging from 0 to 1.

Very diverse recommendations are more likely to engage the user. Less diverse recommendations may just be recommending very closely related movies, like sequels of movies that the user is most likely to have already seen.

#### Novelty

How novel the Top-N movies are i.e., how much of the long-tail of less popular movies is the algorithm able to recommend in the Top-N. Found by dividing the mean of the ranks of the Top-N movies by the lowest ranked movie in the training set. Expressed as a percentage in decimal form ranging from 0 to 1. 

Higher values are usually preferred however, very high values may cause a lack of trust in the recommendations from the user and cause them to disengage with the recommendation list.

#### Validation HitRate

How many movies from the Top-N recommendations are in the validation set. Simply the number of the movies from the Top-N in the validation set divided by the number of Top-N movies. Expressed as a percentage in decimal form ranging from 0 to 1. Higher values show more hits per user.

#### LeaveOneOut-CrossValidation HitRate

The number of times the algorithm recommends the movie left out for LOO-CV in the Top-N recommendations. Either 0 or 1 for every user. Higher values show more hits per user.

Difficult to achieve as the algorithm could recommend many movies the user likes, but miss the one movie left out for this validation set. Useful for smaller data sets where training and test data is limited.

#### Cumulative HitRate

Exactly the same as validation hit-rate however, we now limit the hits to only hits with a predicted rating above some threshold rating. This is to avoid giving the algorithm a positive result even if its recommendations are movies it thinks the user won't actually like.

#### Mean Reciprocal HitRate

Weights the Top-N hits by which page they appear on for a paginated recommendation system. Works just like the validation hit-rate, but now splits up hits into groups of ranks, dividing by the group number (or page) it is in. It then sums these together to give the hit-rate. Higher values show more weighted hits per user.

This is useful for system like Netflix where 5 movies are shown to a page. Our algorithm shouldn't be rewarded as highly for only recommending movies the user actually wants to watch on the last page where the user may not even see them.

#### Actual Rating HitRate

A breakdown of the hits per predicted rating given. The distribution shows us whether the algorithm tends to recommend movies it thinks the user will rate highly or not. The values can be normalised for the total movies available for each user in the validation set showing how many of the ratio for each rating it is able to correctly recommend.

A mean value can be extracted from these to show which ratings the average rating given to the hits of the Top-N recommendations. Higher values will therefore be preferred.

## Results

- Basic results
- Testing the rating prediction paramater
- Testing the 'neighbours' parameter
- Testing the threshold rating parameter
- Testing the 'sample' parameter for item-based algorithms
- Testing the 'sample' parameter for user-based algorithms
- Runtimes

### Parameters and setting up the tests

The recommendation system assumes a 'Netflix' style end-user use case where 30 movies are shown to the user in a paginated form of 5 movies per page. We also only want to recommend movies to the user that have a predicted rating of 3.0 or more. As such, the metrics object for testing was instantiated with the parameters, topN=30, moviesPerPage=5, and thresholdRating=3.0.

The algorithms have the following parameters which can be adjusted to affect the quality of movies recommended:
- 'predict': rating prediction algorithm to use. This can be either of the following:
    - 'calc': use the formulaically calculated value which takes the sum of the weighted similarities, divided by the sum of the unweighted similarities, introduces the input user's bias, and limits to a max rating of 5.0. More information on how this works can be found in part 1.
    - 'mean': use the mean values from the train data,
    - 'sims': multiply similarity scores by max possible rating for predicted ratings,
    - 'norm_sims': normalise similarity scores then multiply by max possible rating for predicted ratings,
    - 'rand': randomly generate ratings.

- 'neighbours': the number of most similar neighbours to draw similarities from. This works slightly differently for item- or user-based algorithms. 
    - Item-based: the algorithm will sample the most similar items per item the input user has rated i.e., if neighbours=100, the algorithm samples the top 100 most similar movies to each movie the input user has rated.
    - User-based: the number of most similar users to the input user. 
- 'threshold': the minimum rating from the input user's ratings which will be considered by the algorithm. The algorithm will remove any movies from the input user's ratings below this value.
- 'sample': this parameter has a different use-case depending on whether the algorithm is item- or user-based.
    - Item-based: samples the top-rated movies from the input user
    - User-based: sample the most similar items per item each similar user has rated. Works similarly to the 'neighbours' parameter for item-based algorithms.
    
We will test each of the following parameters separately, controlling the dependant parameters with the following values: 'predict'='calc', 'threshold'=2.0, 'neighbours'=100, 'sample'=100.

A control test of random predictions will also be run as a control which works by generating random similarity scores, independently generated random ratings, and returning the 30 movies with the highest similarity scores.

A complete set of raw data can be found in *output/ML_MetricsRaw.csv*.

### Testing the quality of various rating prediction methods

Four metrics consider the quality of rating predictions: MAE, RMSE, Cumulative HitRate and Actual Rating HitRate. From the data generated, rating predictions seemingly have negligible impact on runtimes so we will not consider those here.

First, let's consider the MAE and RMSE scores.

Next, we'll look at the percentage differences between the Validation HitRates (i.e. the total hits not considering ratings) and the Cumulative HitRate. This will tell us the difference between the movies predicted that the user has watched and which of those movies the ratings prediction algorithm thinks the user will actually like. For the rating predictions to be working perfectly we would expect to see no difference between Validation HR and Cumulative HR.

### Limiting the most similar neighbours considered

The maximum number of neighbours varies for item- and user-based algorithms as being the maximum number of movies and users in the train data respectively. In this case, those values are 10500 movies and 2822 users. Due to computational limitations causing slower runtimes though, we will be considering neighbours up to 200 for every algorithm apart from Item-Based KNN, where slower runtimes mean we can only test up to 100. However, we will see from the results that the largest variance in the data occurs at these lower values, with results beginning to converge at the upper bounds of our test range.

### Reducing the input user's ratings to some threshold rating

For each algorithm, the threshold rating parameter was tested from 0.5 up to 5.0 in increments of 0.5.

### Sampling the top rated movies for item-based algorithms

The maximum value for the sample parameter will be the value set for the maximum user ratings when initially limiting our data set for building the ratings pivot table, in this case 6000. Once again, computational limitations mean we shall only be testing up to 2000 for all algorithms, and up to 200 for the Item-Based KNN algorithm due to its slower runtimes.

### Sampling each similar user's most similar movies for user-based algorithms

## Re-running the algorithms

## Conclusion

- Discussion
- ReRun basic test for LOO-CV at end.
- Talk about bleeding edge algorithms in conclusion